In [1]:
import zipfile, os

zip_path = "/content/archive (3).zip"  # jangan ubah kalau nama sama
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall("/content/dataset")

base_dir = "/content/dataset"
os.listdir(base_dir)


['House_Room_Dataset']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224,224)
batch = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch,
    class_mode='categorical',
    subset='validation'
)

class_names = list(train_gen.class_indices.keys())
class_names


Found 4200 images belonging to 1 classes.
Found 1050 images belonging to 1 classes.


['House_Room_Dataset']

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

cnn = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.3),
    Dense(train_gen.num_classes,activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_cnn = cnn.fit(train_gen, validation_data=val_gen, epochs=10)
cnn.save("model_cnn_base.h5")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


132/132 ━━━━━━━━━━━━━━━━━━━━ 472s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 528s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 447s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
 54/132 ━━━━━━━━━━━━━━━━━━━━ 4:17 3s/step - accuracy: 1.0000 - loss: 0.0000e+00

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

def evaluate_model(model, generator, history, name):
    # accuracy curve
    plt.figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{name} Accuracy')
    plt.legend(['Train','Val'])
    plt.show()

    # loss curve
    plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{name} Loss')
    plt.legend(['Train','Val'])
    plt.show()

    # predictions
    preds = model.predict(generator)
    y_true = generator.classes
    y_pred = np.argmax(preds, axis=1)

    print("\nClassification Report")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix — {name}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.show()


In [ ]:
evaluate_model(cnn, val_gen, history_cnn, "CNN Base")


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
out = Dense(train_gen.num_classes, activation='softmax')(x)

mobilenet = Model(inputs=base.input, outputs=out)
mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_mobilenet = mobilenet.fit(train_gen, validation_data=val_gen, epochs=8)
evaluate_model(mobilenet, val_gen, history_mobilenet, "MobileNetV2")
mobilenet.save("model_mobilenet.h5")


In [ ]:
from tensorflow.keras.applications import EfficientNetB0

base2 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base2.trainable = False

x = GlobalAveragePooling2D()(base2.output)
x = Dense(128, activation='relu')(x)
out = Dense(train_gen.num_classes, activation='softmax')(x)

effnet = Model(inputs=base2.input, outputs=out)
effnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_eff = effnet.fit(train_gen, validation_data=val_gen, epochs=8)
evaluate_model(effnet, val_gen, history_eff, "EfficientNetB0")
effnet.save("model_efficientnet.h5")


In [ ]:
import pandas as pd

results = {
    "Model": ["CNN Base","MobileNetV2","EfficientNetB0"],
    "Best Val Accuracy": [
        max(history_cnn.history['val_accuracy']),
        max(history_mobilenet.history['val_accuracy']),
        max(history_eff.history['val_accuracy'])
    ]
}

pd.DataFrame(results)
